In [79]:
'''
    Transformer go boom boom attention
    There is also a version of this written in CRC server to make use of GPU cluster :)
    
    Better data generation, just generate all 3^3 combinations possible
    Plot loss vs epoch
    Try network for different points over training
    Separate testing data 
    
    Pursue if victory:
       Try what model does over data it never saw in training i.e. does it learn a general function?
           if not --> it just is cheating and memorizes
           
    Show probability going from probability density --> number prediction (!)
    
    Try doing custom embedding that can work on floats, not just on integer (it would be just a linear nn)
    
    DO POSITIONAL ENCODING
'''

import numpy as np
import torch
import torch.nn as nn
import time
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [154]:
class TransformerModel(nn.Module):
    def __init__(self, n_token, d_model, n_head, n_layers):
        super().__init__()
        
        self.d_model = d_model
        self.embedding = nn.Embedding(n_token, d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=n_head,
            num_encoder_layers=n_layers,
            num_decoder_layers=n_layers)
        self.out = nn.Linear(d_model, n_token) # Learned linear at the end where output of decoder is run through
        
    def forward(self, src, tgt):
        
        src = self.embedding(src) * math.sqrt(self.d_model)
        tgt = self.embedding(tgt) * math.sqrt(self.d_model)

        print(src.size())
        src = src.permute(1, 0, 2)
        tgt = tgt.permute(1, 0, 2)
        
        transformer_output = self.transformer(src, tgt)
        output = self.out(transformer_output)
        print(f'Before permute:\n Size: {output.size()}\nTensor: {output}')
        
        output = output.permute(1, 0, 2) # Permute so that batch num is first dimension again
        #print(f'After permute:\n Size: {output.size()}\nTensor: {output}')
        
        return output

In [155]:
# Let's define an instance :)
d_model = 5 # Let's try embedding brotha
n_token = 7 # Only do 0, 1, 2?
n_head = 1 # Remember that d_model=d_embedding must be divisible by number of n_head
n_layers = 2

model = TransformerModel(n_token, d_model, n_head, n_layers)

print(f'The number of tokens are {n_token}, six categories to calculate probability distributions over (i.e. 0, 1, 2, 3, 4 + sos + eos)')

# Check da dimension
src = torch.randint(size=(2, 3), low=0, high=2)
tgt = src

output = model(src, src)
print(f'src:\n  Size: {src.size()}\n  Tensor: {src}')
print(f'tgt:\n  Size: {tgt.size()}\n  Tensor: {tgt}')
print(f'output:\n  Size: {output.size()}\n  Tensor: {output}\n')

src = torch.randint(size=(2, 3), low=0, high=2)
tgt = src
tgt_input = tgt[:, :-1]
tgt_expected = tgt[:, 1:]
output = model(src, tgt_input)

print('----------------------------------------------------------------------------')
print(f'src:\n  Size: {src.size()}\n  Tensor: {src}')
print(f'tgt_input:\n  Size: {tgt_input.size()}\n  Tensor: {tgt_input}')
print(f'output:\n  Size: {output.size()}\n  Tensor: {output}')

The number of tokens are 7, six categories to calculate probability distributions over (i.e. 0, 1, 2, 3, 4 + sos + eos)
torch.Size([2, 3, 5])
Before permute:
 Size: torch.Size([3, 2, 7])
Tensor: tensor([[[-0.1093,  0.4623, -0.2743,  0.1649,  0.1041, -0.5462,  1.2153],
         [-0.2552,  0.5584,  0.0601,  0.0748,  0.0484, -1.0011,  1.3124]],

        [[-1.0152,  0.2020,  0.2753,  0.6513, -0.8838, -0.3868,  0.9469],
         [-0.7572,  0.4402,  0.0965,  0.5907, -0.2909, -0.8904,  1.2949]],

        [[-0.6724,  0.1188, -0.0306,  0.5339, -0.8580,  0.0553,  0.9333],
         [-0.2526,  0.5380,  0.0575,  0.0960, -0.0371, -0.9735,  1.3748]]],
       grad_fn=<AddBackward0>)
src:
  Size: torch.Size([2, 3])
  Tensor: tensor([[0, 1, 1],
        [0, 1, 0]])
tgt:
  Size: torch.Size([2, 3])
  Tensor: tensor([[0, 1, 1],
        [0, 1, 0]])
output:
  Size: torch.Size([2, 3, 7])
  Tensor: tensor([[[-0.1093,  0.4623, -0.2743,  0.1649,  0.1041, -0.5462,  1.2153],
         [-1.0152,  0.2020,  0.2753,  0.

In [156]:
# Let's make some 1s and 0s 
def construct_data(size, seq_len):
    SOS_token = 5
    EOS_token = 6
    data = np.random.randint(size = (size, seq_len), low=0, high=2)
    target = np.empty((size, seq_len))
    data = torch.tensor(data).to()
    target = torch.tensor(target)
    target[:, 0] = data[:, 1] + data[:, 2]
    target[:, 1] = data[:, 0] + data[:, 2]
    target[:, 2] = data[:, 0] + data[:, 1]
    SOS_ = torch.ones((size, 1)) * SOS_token
    EOS_ = torch.ones((size, 1)) * EOS_token
    
    data = torch.cat((SOS_, data, EOS_), 1)
    target = torch.cat((SOS_, target, EOS_), 1)
    
    return data, target

def batchify(data, target):
    numMiniBatch = int(math.floor(data.shape[0]/100.0))
    inputMiniBatches = data.chunk(numMiniBatch)
    outputMiniBatches = target.chunk(numMiniBatch)
    
    return numMiniBatch, inputMiniBatches, outputMiniBatches

In [157]:
seq_len = 3
size = 10000
data, target = construct_data(size, seq_len)

# Just for confirmation to see we constructed our data the way we were supposed to
print(f' An instance of the input sequence at the 0th index:\n{data[0, :]}')
print(f' An instance of the target sequence at the 0th index:\n{target[0, :]}')

# Batch
batch_num, data_batched, target_batched = batchify(data, target)

# Let's check that batching works properly
print(data_batched[0][0])
print(target_batched[0][0])

 An instance of the input sequence at the 0th index:
tensor([5., 1., 1., 0., 6.])
 An instance of the target sequence at the 0th index:
tensor([5., 1., 1., 2., 6.], dtype=torch.float64)
tensor([5., 1., 1., 0., 6.])
tensor([5., 1., 1., 2., 6.], dtype=torch.float64)


In [162]:
# Train train choo choo, simply and absolutely the best form of transportation

opt = torch.optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()

def train(model, batch_num, data_batched, target_batched):
    model.train()
    total_loss = 0
    start_time = time.time()
    for batch in range(batch_num):
        data = data_batched[batch].type(torch.long)
        target = target_batched[batch].type(torch.long)
          
        target_in = target[:, :-1]
        target_expected = target[:, 1:]
        
        pred = model(data, target_in)

        pred = pred.permute(0, 2, 1) 

        loss = (loss_fn(pred, target_expected)).type(torch.float)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss
    
def train_loop(model, n_epochs, batch_num, data_batched, target_batched):
    for i in range(n_epochs):
        loss = train(model, batch_num, data_batched, target_batched)
        if i % 10 == 0:
            print(f'Epoch: {i}\nTotal Loss: {loss}')
            print(f'----------------------------------')

In [163]:
n_epochs = 200
train_loop(model, n_epochs, batch_num, data_batched, target_batched)

torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[-1.3612,  2.7335,  2.9362,  ..., -2.8607, -2.1814,  2.1626],
         [-0.2127,  3.5005,  2.4470,  ..., -3.1419, -2.8231,  2.5709],
         [ 3.0425,  3.9188,  0.0778,  ..., -2.1413, -2.6699,  1.7543],
         ...,
         [ 2.7385,  4.1096,  0.4525,  ..., -2.5018, -2.9331,  2.1103],
         [-0.9097,  3.1036,  2.7908,  ..., -2.9073, -2.4042,  2.1566],
         [-0.0374,  3.6129,  2.3232,  ..., -3.2041, -2.9925,  2.7325]],

        [[-1.2371,  2.7947,  2.7903,  ..., -2.9982, -2.4223,  2.5152],
         [-0.7377,  2.9434,  2.0754,  ..., -2.6899, -2.8326,  2.6060],
         [ 3.1428,  3.2876, -0.9886,  ..., -2.3103, -3.3648,  3.3460],
         ...,
         [ 3.3914,  3.0775, -1.2933,  ..., -2.1006, -3.1480,  3.1940],
         [-0.9706,  2.9051,  2.8065,  ..., -2.9156, -2.2243,  2.1716],
         [-1.1555,  2.7478,  2.8634,  ..., -2.8839, -2.1213,  2.1553]],

        [[-1.3795,  2.7012,  2.9654, 

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[ 3.9172,  2.1658, -2.2436,  ..., -1.0186, -2.0938,  2.1240],
         [ 0.1154,  3.5539,  2.3393,  ..., -2.8166, -2.4746,  1.9127],
         [ 3.8863,  3.9211, -1.0255,  ..., -1.9941, -3.1939,  2.3711],
         ...,
         [-0.9846,  2.9691,  2.7737,  ..., -3.0327, -2.4232,  2.4334],
         [-1.4886,  2.7173,  2.9798,  ..., -2.8148, -2.1822,  2.1139],
         [ 3.7525,  4.1067, -0.7115,  ..., -2.1967, -3.2615,  2.4117]],

        [[ 3.6043,  1.8050, -2.1944,  ..., -1.0360, -1.9283,  2.2754],
         [ 1.3792,  4.2567,  1.5302,  ..., -2.9117, -3.2120,  2.3395],
         [ 3.2802,  3.6705, -0.8143,  ..., -2.4929, -3.5523,  3.3642],
         ...,
         [-1.6298,  2.5746,  3.0537,  ..., -2.7228, -1.9918,  1.9401],
         [ 0.8027,  4.0550,  1.9323,  ..., -3.0252, -3.0767,  2.3855],
         [ 3.63

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[-0.6117,  3.2382,  2.7155,  ..., -2.9513, -2.4663,  2.1788],
         [-1.4550,  2.3593,  2.9094,  ..., -2.8467, -1.9245,  2.2427],
         [-2.2280,  2.0024,  3.0244,  ..., -2.5261, -1.8436,  2.0496],
         ...,
         [ 3.1432,  3.9924,  0.0532,  ..., -2.2556, -2.8216,  1.9453],
         [-0.8574,  3.0769,  2.7520,  ..., -3.0641, -2.5443,  2.4867],
         [-0.4773,  3.2759,  2.6627,  ..., -2.9084, -2.4230,  2.0818]],

        [[-0.4982,  3.2166,  2.6102,  ..., -3.1014, -2.5844,  2.4991],
         [-1.2599,  2.6810,  2.8382,  ..., -3.0054, -2.3137,  2.5135],
         [-2.1476,  2.1393,  3.1030,  ..., -2.3930, -1.7253,  1.6671],
         ...,
         [ 1.1803,  3.4732,  1.2083,  ..., -3.2777, -3.2818,  3.5266],
         [-0.7788,  3.0392,  2.6425,  ..., -3.1445, -2.6306,  2.7147],
         [-0.81

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[-0.9157,  3.1004,  2.8477,  ..., -2.9435, -2.4228,  2.2035],
         [ 3.9060,  2.0547, -2.2207,  ..., -0.4178, -1.5097,  1.0258],
         [-1.4545,  2.6010,  3.0379,  ..., -2.8183, -2.0245,  2.0636],
         ...,
         [ 2.6266,  4.3703,  0.6148,  ..., -2.8644, -3.4041,  2.6481],
         [ 3.8837,  2.0216, -2.3601,  ..., -0.7649, -1.9227,  1.8375],
         [ 2.1238,  4.3083,  0.9594,  ..., -2.6307, -3.1828,  2.1327]],

        [[-0.7042,  3.1672,  2.6603,  ..., -3.1559, -2.6979,  2.6834],
         [ 2.3606,  1.8221, -1.0745,  ..., -1.9031, -2.2728,  3.1743],
         [-1.5286,  2.4857,  2.5956,  ..., -2.7495, -2.4767,  2.5922],
         ...,
         [ 3.4029,  3.4628, -1.0453,  ..., -2.3379, -3.4251,  3.3184],
         [ 3.8217,  2.3558, -1.8357,  ..., -1.4713, -2.1953,  2.4405],
         [ 2.60

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[ 2.6452,  4.4444,  0.5685,  ..., -2.9381, -3.5753,  2.8362],
         [-1.1174,  2.9943,  2.9412,  ..., -2.7907, -2.2515,  1.9518],
         [-1.7381,  2.4384,  3.1039,  ..., -2.8015, -2.0001,  2.1438],
         ...,
         [-2.9590,  1.0319,  3.1759,  ..., -2.1770, -1.0062,  1.6925],
         [ 3.2880,  3.9416, -0.4477,  ..., -1.8341, -2.9295,  1.7568],
         [-1.8565,  2.3503,  3.0959,  ..., -2.7958, -2.0162,  2.2231]],

        [[ 2.9285,  3.7920, -0.4322,  ..., -2.7204, -3.7149,  3.5411],
         [-0.2911,  3.4892,  2.5679,  ..., -3.1075, -2.7556,  2.4627],
         [-1.5117,  2.5426,  2.9762,  ..., -2.9351, -2.1699,  2.3972],
         ...,
         [-2.8772,  1.3465,  3.2641,  ..., -2.1246, -1.1295,  1.4565],
         [ 3.0050,  3.3186, -0.7617,  ..., -2.5410, -3.4116,  3.5650],
         [-1.51

...

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[-2.8300,  1.5056,  3.4243,  ..., -2.6798, -1.7327,  2.5898],
         [ 3.4802,  4.5776, -0.0420,  ..., -2.6387, -3.4222,  2.4199],
         [ 3.0182,  4.3005,  0.2096,  ..., -2.2170, -2.8761,  1.6140],
         ...,
         [-0.2409,  3.8348,  2.7825,  ..., -3.1509, -2.9086,  2.3925],
         [ 4.3493,  2.8059, -2.2639,  ..., -1.5045, -2.7284,  2.6185],
         [ 3.2959,  4.3023, -0.2779,  ..., -2.0951, -3.1334,  1.8496]],

        [[-3.1555,  1.4114,  3.6662,  ..., -2.4297, -1.4056,  2.0246],
         [ 3.2716,  4.4527, -0.0487,  ..., -3.1644, -3.9584,  3.6099],
         [ 3.5445,  3.9188, -0.8637,  ..., -2.7183, -3.8284,  3.6683],
         ...,
         [-0.1405,  3.4156,  1.8826,  ..., -3.1776, -3.5631,  3.6334],
         [ 3.7264,  2.2503, -1.9294,  ..., -1.8048, -2.5916,  3.2051],
         [ 3.16

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[ 4.3154,  1.6977, -2.8139,  ..., -0.4727, -1.4799,  1.3986],
         [-0.7622,  3.5854,  3.0259,  ..., -3.2801, -2.9311,  2.7026],
         [ 3.4144,  4.5278, -0.2349,  ..., -2.4178, -3.4786,  2.3184],
         ...,
         [-0.0484,  4.0578,  2.6263,  ..., -3.3126, -3.2850,  2.7720],
         [ 0.1543,  3.9797,  2.5692,  ..., -3.3655, -3.1491,  2.7831],
         [ 3.7934,  4.5168, -0.5409,  ..., -2.4982, -3.5886,  2.6000]],

        [[ 1.9969,  0.4901, -1.4834,  ..., -1.2920, -1.4224,  2.9311],
         [-0.0868,  3.6969,  2.6694,  ..., -3.3570, -2.9569,  2.8084],
         [ 2.8649,  4.0841,  0.0744,  ..., -3.2717, -3.9051,  3.9576],
         ...,
         [-1.5004,  2.8915,  3.2982,  ..., -3.1746, -2.4811,  2.6856],
         [ 0.2465,  4.1117,  2.5078,  ..., -3.3772, -3.2821,  2.8246],
         [ 3.28

pred: torch.Size([100, 4, 7])
target: torch.Size([100, 4])
pred after: torch.Size([100, 7, 4])
torch.Size([100, 5, 5])
Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[ 4.4071,  4.0851, -1.2368,  ..., -2.1048, -3.1470,  2.3016],
         [-2.9292,  1.4760,  3.4930,  ..., -2.6444, -1.6790,  2.5147],
         [ 4.5476,  2.3783, -2.6152,  ..., -0.9352, -2.0824,  1.8356],
         ...,
         [ 4.0124,  1.6496, -2.6802,  ..., -0.8926, -1.9044,  2.2624],
         [ 3.8147,  4.4765, -0.4075,  ..., -2.4723, -3.3298,  2.3261],
         [-2.9236,  1.7334,  3.7018,  ..., -2.5791, -1.5985,  2.0920]],

        [[ 4.0351,  2.6892, -2.0849,  ..., -1.7373, -2.8827,  3.0673],
         [-3.3117,  1.1547,  3.5434,  ..., -2.3646, -1.3971,  2.2044],
         [ 4.2397,  2.0851, -2.4994,  ..., -1.2682, -2.2275,  2.5585],
         ...,
         [ 3.2090,  1.4063, -2.0431,  ..., -1.4246, -2.0576,  3.0327],
         [ 3.6611,  3.3931, -1.4104,  ..., -2.2415, -3.4869,  3.4130],
         [-3.25

Before permute:
 Size: torch.Size([4, 100, 7])
Tensor: tensor([[[-3.2689,  0.9261,  3.5377,  ..., -2.3719, -1.1900,  2.2141],
         [-1.6751,  2.7573,  3.4373,  ..., -3.0660, -2.2942,  2.4696],
         [ 3.5023,  4.3074, -0.2410,  ..., -2.1535, -2.9899,  1.7496],
         ...,
         [-1.5573,  2.9998,  3.3961,  ..., -3.1419, -2.5410,  2.5895],
         [-3.4585,  0.7472,  3.5480,  ..., -2.2369, -1.0505,  2.0858],
         [-1.3383,  3.1048,  3.3402,  ..., -2.8655, -2.2722,  1.9552]],

        [[-3.4328,  0.7544,  3.3183,  ..., -2.1878, -1.2732,  2.2985],
         [-1.5921,  2.7879,  3.4127,  ..., -3.0487, -2.2661,  2.4037],
         [ 1.5205,  3.9267,  1.3819,  ..., -3.5485, -3.5987,  3.7609],
         ...,
         [-1.7731,  2.5808,  3.3547,  ..., -3.1635, -2.4004,  2.8510],
         [-3.4619,  1.1393,  3.6371,  ..., -2.1510, -1.2386,  1.8108],
         [-0.4722,  3.0309,  1.8063,  ..., -2.7439, -3.2288,  3.1973]],

        [[-3.3372,  1.0193,  3.5861,  ..., -2.3671, -1.2736, 

KeyboardInterrupt: 

In [115]:
def predict(model, input_seq, max_length=5, SOS_token=5, EOS_token=6):
    model.eval()
    
    target_input = torch.tensor([[SOS_token]], dtype=torch.long, device=device)

    # Asks model to give only one item, the next thing it thinks is most probable 
    # to continue the sentence
    for i in range(max_length):
        
        pred = model(input_seq, target_input)
        next_item = pred.topk(1)[1].view(-1)[0].item() # num with highest probability, twas view(-1)[-1] before
        next_item = torch.tensor([[next_item]], device=device)

        # Concatenate previous input with predicted best word
        target_input = torch.cat((target_input, next_item), dim=1)

        # Stop if model predicts end of sentence
        if next_item.view(-1).item() == EOS_token:
            break
        elif (len(target_input.view(-1)) >= len(input_seq[0])):
            break

    return target_input.view(-1).tolist()

In [165]:
example = torch.tensor([[5, 1, 1, 1, 6]]).type(torch.long)
result = predict(model, example)
print(f"Example")
print(f"Input: {example.view(-1).tolist()[1:-1]}")
print(f"Continuation: {result[1:-1]}")

example = torch.tensor([[5, 0, 0, 0, 6]]).type(torch.long)
result = predict(model, example)
print(f"Example")
print(f"Input: {example.view(-1).tolist()[1:-1]}")
print(f"Continuation: {result[1:-1]}")

example = torch.tensor([[5, 1, 0, 2, 6]]).type(torch.long)
result = predict(model, example)
print(f"Example")
print(f"Input: {example.view(-1).tolist()[1:-1]}")
print(f"Continuation: {result[1:-1]}")

torch.Size([1, 5, 5])
Before permute:
 Size: torch.Size([1, 1, 7])
Tensor: tensor([[[-2.6484,  1.9779,  3.5695, -1.8861, -2.8080, -1.9466,  2.5078]]],
       grad_fn=<AddBackward0>)
torch.Size([1, 5, 5])
Before permute:
 Size: torch.Size([2, 1, 7])
Tensor: tensor([[[-2.8414,  1.6805,  3.6075, -1.6949, -2.6785, -1.6684,  2.3490]],

        [[-2.6494,  1.9911,  3.4576, -1.8127, -2.7966, -2.0892,  2.6494]]],
       grad_fn=<AddBackward0>)
torch.Size([1, 5, 5])
Before permute:
 Size: torch.Size([3, 1, 7])
Tensor: tensor([[[-2.9311,  1.5419,  3.6023, -1.5876, -2.6211, -1.5697,  2.3138]],

        [[-2.8487,  1.7730,  3.5162, -1.6703, -2.6940, -1.8900,  2.5216]],

        [[-2.8487,  1.7730,  3.5162, -1.6703, -2.6940, -1.8900,  2.5216]]],
       grad_fn=<AddBackward0>)
torch.Size([1, 5, 5])
Before permute:
 Size: torch.Size([4, 1, 7])
Tensor: tensor([[[-2.9623,  1.4876,  3.5949, -1.5425, -2.6012, -1.5365,  2.3094]],

        [[-2.9396,  1.6643,  3.5434, -1.6017, -2.6434, -1.7866,  2.4550]],
